In [9]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
import pickle
from tqdm import tqdm

In [3]:
task_data = pd.read_csv('PD_data_choice.csv')

### BRL

In [5]:
def pd_brl_loglikelihood(sub_data, beta_param, bias, gamma_pos, gamma_neg):
    """
    Bayesian Reinforcement Learning log-likelihood function for Iterative Prisoner's Dilemma
    
    Parameters:
    sub_data: DataFrame with subject data (80 rounds per subject)
    beta_param: softmax temperature parameter for the player
    bias: bias parameter
    gamma_pos: learning rate for cooperative outcomes
    gamma_neg: learning rate for defective outcomes
    
    Returns:
    BayesLL: negative log-likelihood
    """
    
    # Read trial parameters
    trial = sub_data['round_number'].values
    p1_choice = sub_data['p1_choice'].values
    p2_choice = sub_data['p2_choice'].values
    
    # Number of trials
    num_trials = len(trial)
    
    # Initialize priors for each 'player type': uniform probability
    a_init = 1.01
    b_init = 1.01
    
    # Set alpha and beta for each 'player type'
    a = a_init
    b = b_init
    
    # Store mean beta distribution and log likelihood
    mew = np.zeros(num_trials)
    loglike = np.zeros(num_trials)
    
    # Loop over trials
    for t in range(num_trials):
        # Beta distribution to compute chris's probability of cooperating
        mew[t] = a / (a + b)
        
        # Human choice probability
        if p1_choice[t] == 1:  # probability of cooperating
            choice_prob = np.exp(beta_param * mew[t]) / (np.exp(beta_param * mew[t]) + np.exp(beta_param * bias))
        else:  # probability of defecting
            choice_prob = 1 - np.exp(beta_param * mew[t]) / (np.exp(beta_param * mew[t]) + np.exp(beta_param * bias))
        
        # Update strategies based on partner's action
        if p2_choice[t] == 1:
            a += 1  # cooperate
        else:
            b += 1  # defect
        
        a_prev = a
        b_prev = b
        
        # Implement learning rate
        a = a_prev * gamma_pos  # cooperate LR
        b = b_prev * gamma_neg  # defect LR
        
        # Get trial log-likelihood
        loglike[t] = np.log(choice_prob)
    
    # Return negative sum of log-likelihood
    bayes_ll = -np.sum(loglike)
    return bayes_ll

In [10]:
bounds = [(1, 20), (0.1, 1), (0.1, 1), (0.1, 1)]
    
# Initialize results dictionary
params = {
    'subjNum': [],
    'subjBetaParam': [],
    'subjBias': [],
    'subjGammaPos': [],
    'subjGammaNeg': [],
    'subjBayesLL': []
}

# Subject list
sub_list = task_data['ppt_number'].unique()

for sub_num in tqdm(sub_list, desc="Fitting subjects", unit="subject"):
    
    # Get subject data
    sub_data = task_data[task_data['ppt_number'] == sub_num].copy()
    
    # Remove NaNs
    sub_data = sub_data.dropna()
    
    # Set number of iterations/starting points
    n_iter = 20
    
    if not sub_data.empty:  # if we have data for subject to model
        
        results = []
        likelihoods = []
        
        for iteration in range(n_iter):
            # Random initial values
            np.random.seed()
            init_vals = [
                np.random.rand() * 20,                    # beta_param: 0-20
                (np.random.rand() + 0.1) * 0.9,         # bias: 0.1-1
                (np.random.rand() + 0.1) * 0.9,         # gamma_pos: 0.1-1
                (np.random.rand() + 0.1) * 0.9          # gamma_neg: 0.1-1
            ]
            
            # Optimize parameters
            def objective(x):
                return pd_brl_loglikelihood(sub_data, x[0], x[1], x[2], x[3])
            
            try:
                result = minimize(objective, init_vals, bounds=bounds, method='L-BFGS-B')
                results.append(result.x)
                likelihoods.append(result.fun)
            except:
                # If optimization fails, skip this iteration
                continue
        
        if likelihoods:  # If we have valid results
            # Find minimum likelihood
            min_bayes_ll = min(likelihoods)
            best_idx = likelihoods.index(min_bayes_ll)
            
            # Save best fitting parameters
            best_params = results[best_idx]
            beta = best_params[0]
            bias = best_params[1]
            gamma_pos = best_params[2]
            gamma_neg = best_params[3]
            
            # Store results
            params['subjNum'].append(sub_num)
            params['subjBetaParam'].append(beta)
            params['subjBias'].append(bias)
            params['subjGammaPos'].append(gamma_pos)
            params['subjGammaNeg'].append(gamma_neg)
            params['subjBayesLL'].append(min_bayes_ll)
            
            # Save intermediate results
            with open('m1_PD_BRL.pkl', 'wb') as f:
                pickle.dump(params, f)

# Compute AIC
num_params = len(bounds)
params['subjAIC'] = [-2 * ll - 2 * num_params for ll in params['subjBayesLL']]

# Create results DataFrame
results_df = pd.DataFrame({
    'sub_num': params['subjNum'],
    'betaParam': params['subjBetaParam'],
    'bias': params['subjBias'],
    'gammaPos': params['subjGammaPos'],
    'gammaNeg': params['subjGammaNeg'],
    'BayesLL': params['subjBayesLL'],
    'AIC': params['subjAIC']
})

# Save to Excel file
results_df.to_csv('PD_BRL_fitting_results.csv', index=False)

print("Model fitting completed!")

Fitting subjects: 100%|██████████| 142/142 [01:50<00:00,  1.28subject/s]

Model fitting completed!
